In [5]:
import pandas as pd
import glob

# Define the directory where your CSV files are located
directory = '/Users/dhewitt/Data/pps/Modelling/'

# Define the pattern to match the CSV files
file_pattern = directory + 'P*_datEVT.csv'

# Get a list of all CSV files matching the pattern
csv_files = glob.glob(file_pattern)
dfs = []

In [8]:
directory = '/Users/dhewitt/Data/pps/Modelling/'

# Define the pattern to match the CSV files
file_pattern = directory + 'P*_datEVT.csv'

# Get a list of all CSV files matching the pattern
csv_files = glob.glob(file_pattern)

# Create an empty list to store individual DataFrames
dfs = []

# Loop through each CSV file
for csv_file in csv_files:
    # Extract participant ID from the file name
    participant_id = csv_file.split('/')[-1].split('_')[0][1:]
    
    # Load CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Add participant ID column
    df['Participant_ID'] = participant_id
    
    # Append DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into one large DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

In [10]:
combined_df.head()

,latency,type,epoch,Block,Include,pain,Freq_f1_t1_el1,Freq_f1_t1_el2,Freq_f1_t1_el3,Freq_f1_t2_el1,...,Freq_f3_t1_el1,Freq_f3_t1_el2,Freq_f3_t1_el3,Freq_f3_t2_el1,Freq_f3_t2_el2,Freq_f3_t2_el3,Freq_f3_t3_el1,Freq_f3_t3_el2,Freq_f3_t3_el3,Participant_ID
0,1251.0,middle,1,1,1,nPCollNoShock,84.717666,55.937957,38.038957,10.083858,...,47.847792,-4.704442,68.023919,57.380406,4.970013,70.130336,81.745067,80.722153,87.411712,16
1,4251.0,right,2,1,1,PCollShock,1.837269,11.790846,79.662190,-6.334087,...,26.404309,-20.807507,21.021438,43.668933,-9.693051,32.263670,76.975799,13.071432,20.947232,16
2,7251.0,right,3,1,1,PCollShock,-30.558103,38.882345,27.250060,-31.115323,...,36.291080,134.768125,143.989528,48.778112,121.091798,67.535705,75.015480,100.552867,22.057330,16
3,10251.0,left,4,1,1,PCollShock,5.936221,37.020377,63.117695,35.110421,...,39.192777,102.591900,39.395846,43.007762,140.517548,53.146010,57.255875,66.786078,77.292349,16
4,13251.0,left,5,1,1,PCollNoShock,-5.096196,25.909149,53.337064,-9.571798,...,19.280457,69.328385,48.709259,28.972982,84.067207,47.454706,38.552883,99.940679,55.415618,16


In [22]:
def count_include_values(group):
    return group['Include'].value_counts()

# Group the DataFrame by 'Participant_ID', 'type', 'block', and 'pain', and apply the count_include_values function
counts = combined_df.groupby(['Participant_ID', 'type', 'Block']).apply(count_include_values).unstack(fill_value=0)

# Reset index to make the DataFrame flat
counts.reset_index(inplace=True)

In [23]:
counts

,Participant_ID,type,Block,0,1,2
0,02,left,1,0,22,2
1,02,left,2,0,36,0
2,02,left,3,0,20,4
3,02,left,4,0,34,2
4,02,middle,1,0,22,2
...,...,...,...,...,...,...
307,30,middle,4,1,35,0
308,30,right,1,0,24,0
309,30,right,2,0,36,0
310,30,right,3,0,24,0


In [37]:
totals = counts.groupby(['Participant_ID','Block','type']).sum()

In [38]:
totals.to_csv('/Users/dhewitt/Data/pps/Modelling/totals.csv')

In [39]:
# Calculate the mean of 0, 1, and 2 values for each block
mean_values = totals.groupby('Block').mean()
print(mean_values)

# Calculate the standard deviation of 0, 1, and 2 values for each block
std_values = totals.groupby('Block').std()
print(std_values)

              0          1         2
Block                               
1      0.500000  23.384615  0.128205
2      0.987179  34.923077  0.076923
3      0.820513  22.987179  0.205128
4      1.076923  34.448718  0.448718
              0         1         2
Block                              
1      1.148121  1.240347  0.437198
2      1.284206  1.365334  0.268194
3      1.365090  1.463793  0.811588
4      1.181780  1.525944  0.906648


In [40]:
# Calculate the mean of 0, 1, and 2 values for each block
mean_values = totals.mean()
print(mean_values)

# Calculate the standard deviation of 0, 1, and 2 values for each block
std_values = totals.std()
print(std_values)

0     0.846154
1    28.935897
2     0.214744
dtype: float64
0    1.261112
1    5.930118
2    0.672406
dtype: float64


In [41]:
# Filter out blocks 1 and 2 as conditioning blocks
conditioning = totals[totals.index.get_level_values('Block').isin([1, 3])]

# Calculate the overall mean and standard deviation for conditioning blocks
conditioning_mean = conditioning.mean()
conditioning_std = conditioning.std()
print(conditioning_mean)
print(conditioning_std)

# Filter out blocks 2 and 3 as extinction blocks
extinction = totals[totals.index.get_level_values('Block').isin([2, 4])]

# Calculate the overall mean and standard deviation for extinction blocks
extinction_mean = extinction.mean()
extinction_std = extinction.std()
print(extinction_mean)
print(extinction_std)

# Now conditioning_mean, conditioning_std, extinction_mean, and extinction_std contain the overall mean and standard deviation for conditioning and extinction blocks


0     0.660256
1    23.185897
2     0.166667
dtype: float64
0    1.267442
1    1.366911
2    0.650889
dtype: float64
0     1.032051
1    34.685897
2     0.262821
dtype: float64
0    1.230892
1    1.462673
2    0.692002
dtype: float64
